In [3]:
# ETL com Python - 04.07.2022 - 20h11


# Importando a biblioteca Pandas
import pandas as pd

# Importando a biblioteca Pandera
import pandera as pa

In [4]:
# Usando o método read_csv (nome do arquivo como parametro) e retorna um objeto de dados
df = pd.read_csv('ocorrencia_2015.csv', parse_dates=['ocorrencia_dia'], dayfirst = True)

# Convertendo a coluna ocorrencia_dia para tipo datetime
# dayfirst = True - para a data vir primeiro
df["ocorrencia_dia"] = pd.to_datetime(df["ocorrencia_dia"], errors = "coerce", format = "%d/%m/%Y")
df["total_recomendacoes"] = pd.to_numeric(df["total_recomendacoes"])

# Saida do dataframe - usando a funcao head para imprimir as primeiras 10 linhas
# se nao colocar parametro, a funcao head imprime as 5 primeiras linhas.
df.head(10)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,Unnamed: 9,Unnamed: 10
0,52242,52242,INCIDENTE,PORTO ALEGRE,RS,SBPA,2012-01-05,20:27:00,0,NaN,NaN
1,45331,45331,ACIDENTE,GUARULHOS,SP,SBGR,2012-01-06,13:44:00,3,NaN,NaN
2,45333,45333,ACIDENTE,VIAMÃO,RS,****,2012-01-06,13:00:00,0,NaN,NaN
3,45401,45401,ACIDENTE,SÃO SEBASTIÃO,SP,****,2012-01-06,17:00:00,0,NaN,NaN
4,45407,45407,ACIDENTE,SÃO SEPÉ,RS,****,2012-01-06,16:30:00,0,NaN,NaN
5,52243,52243,INCIDENTE,UBATUBA,SP,****,2012-01-06,14:30:00,0,NaN,NaN
6,50713,50713,INCIDENTE GRAVE,CAMPINAS,SP,SDAI,2012-01-07,18:15:00,0,NaN,NaN
7,45334,45334,INCIDENTE,BELÉM,PA,SBBE,2012-01-08,19:12:00,0,NaN,NaN
8,45391,45391,ACIDENTE,CONCEIÇÃO DAS ALAGOAS,MG,****,2012-01-08,16:00:00,0,NaN,NaN
9,52244,52244,INCIDENTE,UBERLÂNDIA,MG,SBUL,2012-01-08,22:13:00,0,NaN,NaN


In [5]:
# Checando os tipos de dados de cada coluna
df.dtypes

codigo_ocorrencia                    int64
codigo_ocorrencia2                   int64
ocorrencia_classificacao            object
ocorrencia_cidade                   object
ocorrencia_uf                       object
ocorrencia_aerodromo                object
ocorrencia_dia              datetime64[ns]
ocorrencia_hora                     object
total_recomendacoes                  int64
Unnamed: 9                         float64
Unnamed: 10                         object
dtype: object

In [ ]:
df.ocorrencia_dia.dt.day_of_week

In [8]:
# Montando o esquema para validar cada Coluna com o seu tipo de dado.
# usando a funcao DataFrameSchema da biblioteca Pandera
schema = pa.DataFrameSchema(
    columns = {
        'codigo_ocorrencia': pa.Column(pa.Int),
        'codigo_ocorrencia2': pa.Column(pa.Int),
        'ocorrencia_classificacao': pa.Column(pa.String),
        'ocorrencia_cidade': pa.Column(pa.String),
        'ocorrencia_uf': pa.Column(pa.String, pa.Check.str_length(2, 2), nullable = True), # tamanho minimo e maximo da coluna
        'ocorrencia_aerodromo': pa.Column(pa.String),
        'ocorrencia_dia': pa.Column(pa.DateTime, nullable = True),
        'ocorrencia_hora': pa.Column(pa.String, pa.Check.str_matches(r'ˆ([0-1]?[0-9]|[2][0-3]):([0-5][0-9])(:[0-5][0-9])?$'), nullable = True),
        'total_recomendacoes': pa.Column(pa.Int)
    }
)


# coluna {
  #  'coluna': pa.Column(pa.Tipodedadodacoluna)
# nullable = True, permitir que valores nulos estejam na coluna especificada.
# funcao Check - restricao - uso de regular expressions para validar o campo hora.
# }


In [26]:
# Montando o esquema para validacao do DataFrame completo
# Se o DataFrame for exibido, a validacao foi feita com sucesso.
schema.validate(df)

SchemaError: <Schema Column(name=ocorrencia_uf, type=DataType(str))> failed element-wise validator 0:
<Check str_length: str_length(2, 2)>
failure cases:
    index failure_case
0    1794    890274583
1    3334        82439
2    3366      6372222
3    4346         6530
4    4387      1408920
5    4417         4780
6    4538         3705
7    4649       961388
8    4728          ***
9    4757       605000
10   4776        20083
11   4794       469226
12   4888         9186
13   4890         2528
14   5062        48305
15   5070        89500

In [10]:
# Acessando individualmente cada item contido na interseçao linha coluna
# objeto_dataframe.loc[linha, "nome_coluna]
df.loc[1099, "ocorrencia_uf"]

'***'

In [27]:
# Obtendo somente a linha - informando o label da linha
df.loc[1099]

codigo_ocorrencia                          49474
codigo_ocorrencia2                         49474
ocorrencia_classificacao                ACIDENTE
ocorrencia_cidade           ÁGUAS INTERNACIONAIS
ocorrencia_uf                                 AI
ocorrencia_aerodromo                        ****
ocorrencia_dia               2013-09-02 00:00:00
ocorrencia_hora                         02:54:00
total_recomendacoes                            0
Unnamed: 9                                   NaN
Unnamed: 10                                  NaN
Name: 1099, dtype: object

In [12]:
# Fazendo slicing com o metodo loc
df.loc[1099:1100]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,Unnamed: 9,Unnamed: 10
1099,49474,49474,ACIDENTE,ÁGUAS INTERNACIONAIS,***,****,2013-09-02,02:54:00,0,NaN,NaN
1100,49611,49611,ACIDENTE,BELÉM,PA,****,2013-09-03,17:45:00,1,NaN,NaN


In [13]:
# Extraindo somente linhas especificas - passar as linhas desejadas dentro de uma lista
df.loc[[10,1099]]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,Unnamed: 9,Unnamed: 10
10,45332,45332,ACIDENTE,VIAMÃO,RS,****,2012-01-09,13:30:00,0,NaN,NaN
1099,49474,49474,ACIDENTE,ÁGUAS INTERNACIONAIS,***,****,2013-09-02,02:54:00,0,NaN,NaN


In [14]:
# Somente colunas - informar na linha, : - primeiro e ultimo 'indice
df.loc[:, "ocorrencia_cidade"]

0        PORTO ALEGRE
1           GUARULHOS
2              VIAMÃO
3       SÃO SEBASTIÃO
4            SÃO SEPÉ
            ...      
5162            JATAÍ
5163          MARACAÍ
5164    NOVO HAMBURGO
5165         CURITIBA
5166        PETROLINA
Name: ocorrencia_cidade, Length: 5167, dtype: object

In [15]:
# VErificando se uma coluna, tem informacoes unicas, que nao se repetem - retorna booleano
df.codigo_ocorrencia.is_unique

True

In [16]:
# Setando uma coluna como 'indice' do dataframe
# Atributo inplace, para que seja atualizado em qualquer mostragem do dataframe posterior.
df.set_index("codigo_ocorrencia", inplace = True)

In [17]:
df.head()

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,Unnamed: 9,Unnamed: 10
codigo_ocorrencia,,,,,,,,,,
52242,52242,INCIDENTE,PORTO ALEGRE,RS,SBPA,2012-01-05,20:27:00,0,NaN,NaN
45331,45331,ACIDENTE,GUARULHOS,SP,SBGR,2012-01-06,13:44:00,3,NaN,NaN
45333,45333,ACIDENTE,VIAMÃO,RS,****,2012-01-06,13:00:00,0,NaN,NaN
45401,45401,ACIDENTE,SÃO SEBASTIÃO,SP,****,2012-01-06,17:00:00,0,NaN,NaN
45407,45407,ACIDENTE,SÃO SEPÉ,RS,****,2012-01-06,16:30:00,0,NaN,NaN


In [19]:
# Removendo a coluna setada como 'indice' do dataframe
df.reset_index(drop = True, inplace = True)
df.head()

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,Unnamed: 9,Unnamed: 10
0,52242,52242,INCIDENTE,PORTO ALEGRE,RS,SBPA,2012-01-05,20:27:00,0,NaN,NaN
1,45331,45331,ACIDENTE,GUARULHOS,SP,SBGR,2012-01-06,13:44:00,3,NaN,NaN
2,45333,45333,ACIDENTE,VIAMÃO,RS,****,2012-01-06,13:00:00,0,NaN,NaN
3,45401,45401,ACIDENTE,SÃO SEBASTIÃO,SP,****,2012-01-06,17:00:00,0,NaN,NaN
4,45407,45407,ACIDENTE,SÃO SEPÉ,RS,****,2012-01-06,16:30:00,0,NaN,NaN


In [24]:
# Alterando dados de uma unica intersecao linha coluna
df.loc[1099, "ocorrencia_uf"] = "AI"

In [25]:
df.loc[1099]

codigo_ocorrencia                          49474
codigo_ocorrencia2                         49474
ocorrencia_classificacao                ACIDENTE
ocorrencia_cidade           ÁGUAS INTERNACIONAIS
ocorrencia_uf                                 AI
ocorrencia_aerodromo                        ****
ocorrencia_dia               2013-09-02 00:00:00
ocorrencia_hora                         02:54:00
total_recomendacoes                            0
Unnamed: 9                                   NaN
Unnamed: 10                                  NaN
Name: 1099, dtype: object

In [28]:
# criando uma coluna de backup
df["ocorrencia_uf_bkp"] = df["ocorrencia_uf"]

In [29]:
# Impressao do DataSet
df

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,Unnamed: 9,Unnamed: 10,ocorrencia_uf_bkp
0,52242,52242,INCIDENTE,PORTO ALEGRE,RS,SBPA,2012-01-05,20:27:00,0,NaN,NaN,RS
1,45331,45331,ACIDENTE,GUARULHOS,SP,SBGR,2012-01-06,13:44:00,3,NaN,NaN,SP
2,45333,45333,ACIDENTE,VIAMÃO,RS,****,2012-01-06,13:00:00,0,NaN,NaN,RS
3,45401,45401,ACIDENTE,SÃO SEBASTIÃO,SP,****,2012-01-06,17:00:00,0,NaN,NaN,SP
4,45407,45407,ACIDENTE,SÃO SEPÉ,RS,****,2012-01-06,16:30:00,0,NaN,NaN,RS
...,...,...,...,...,...,...,...,...,...,...,...,...
5162,80458,80458,ACIDENTE,JATAÍ,GO,####,2021-12-30,20:30:00,0,NaN,NaN,GO
5163,80452,80452,ACIDENTE,MARACAÍ,SP,*****,2021-12-31,09:30:00,0,NaN,NaN,SP
5164,80457,80457,INCIDENTE GRAVE,NOVO HAMBURGO,RS,SSNH,2021-12-31,11:59:00,0,NaN,NaN,RS
5165,80460,80460,INCIDENTE,CURITIBA,PR,SBBI,2021-12-31,15:12:00,0,NaN,NaN,PR


In [30]:
# Alterando dados em uma determinada coluna, de acordo com alguma condica"
# df.loc[procurando onde se tem na uf, SP, [na coluna ocorrencia_clas]] = 'Alterar para grave'
df.loc[df.ocorrencia_uf == "SP", ["ocorrencia_classificacao"]] = "GRAVE"

In [31]:
df.loc[df.ocorrencia_uf == "SP"]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,Unnamed: 9,Unnamed: 10,ocorrencia_uf_bkp
1,45331,45331,GRAVE,GUARULHOS,SP,SBGR,2012-01-06,13:44:00,3,NaN,NaN,SP
3,45401,45401,GRAVE,SÃO SEBASTIÃO,SP,****,2012-01-06,17:00:00,0,NaN,NaN,SP
5,52243,52243,GRAVE,UBATUBA,SP,****,2012-01-06,14:30:00,0,NaN,NaN,SP
6,50713,50713,GRAVE,CAMPINAS,SP,SDAI,2012-01-07,18:15:00,0,NaN,NaN,SP
13,45408,45408,GRAVE,ELDORADO,SP,****,2012-01-11,13:45:00,1,NaN,NaN,SP
...,...,...,...,...,...,...,...,...,...,...,...,...
5148,80441,80441,GRAVE,BRAGANÇA PAULISTA,SP,SDVH,2021-12-20,18:15:00,0,NaN,NaN,SP
5157,80453,80453,GRAVE,CAMPINAS,SP,SBKP,2021-12-29,09:00:00,0,NaN,NaN,SP
5158,80454,80454,GRAVE,ARAÇATUBA,SP,SBAU,2021-12-29,21:35:00,0,NaN,NaN,SP
5161,80456,80456,GRAVE,SÃO PAULO,SP,SBSP,2021-12-30,13:15:00,0,NaN,NaN,SP
